# Predict tags on StackOverflow with linear models

### Set working dir

In [30]:
!pip3 install --user underthesea

In [31]:
!pip3 install --user nltk

In [32]:
!pip3 install --user bs4

In [33]:
!pip3 install --user pandas

In [34]:
!pip3 install --user matplotlib

In [35]:
from shutil import copyfile

# drive_dir = './drive/My Drive/AutoTaggingColab/Github @tranphuquy19/'
working_dir = "C:\\Users\\Christian\\Documents\\myProjects\\SeniorProject\\multitag\\data\\final\\"

db_name = 'development.db'
title_index = 0
tag_index = 1
html_index = 2

In [36]:
# copyfile(drive_dir + 'data/' + db_name, working_dir + db_name)

In [37]:
import sqlite3 as lite
import sys
import os

con = None 

# try: 
#   path = working_dir + db_name
#   con = lite.connect(path)

#   cur = con.cursor()
#   cur.execute("SELECT title, tags, htmlContent FROM articleModels WHERE tags != 'none' LIMIT 3;")

#   # data = cur.fetchone()
#   rows = cur.fetchall()
#   for row in rows:
#     soup = BeautifulSoup(row[html_index])
#     text = soup.get_text().replace('\n ',' ') 
#     print(text)

# except lite.Error as e:
#   print(e)
#   sys.exit(1)
# finally:
#   if con:
#     con.close()

### Text preprocessing

In [38]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"
 
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):

    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)



In [39]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


#### Copy Vietnamese StopWords files (mounted GDrive)

In [40]:
# ! cp './drive/My Drive/AutoTaggingColab/Github @tranphuquy19/data/vietnamese' '/root/nltk_data/corpora/stopwords/vietnamese'

In [41]:
from ast import literal_eval
import pandas as pd
import numpy as np

In [42]:
def read_data(filename):
    data = pd.read_csv(filename)
    data['tags'] = data['tags'].apply(literal_eval)
    return data

In [43]:
# train = read_data(drive_dir + 'data/train.tsv')
# validation = read_data( drive_dir + 'data/validation.tsv')
# test = pd.read_csv(drive_dir + 'data/test.tsv', sep='\t')

train = read_data(working_dir + 'vi_blog_train.csv')
validation = read_data( working_dir + 'vi_blog_validation.csv')
test = pd.read_csv(working_dir + 'vi_blog_test.csv')

In [44]:
# train = train[:15000]
train.head()

,title,tags
0,chào_mừng trở_lại blog duthaho tết_nhất dí thi...,[git]
1,hôm_nay chạy ứng_dụng docker đơn_giản ví_dụ no...,[docker]
2,thực_chất câu_chuyện anh_chàng buôn chuối viết...,[javascript]
3,đẹp trời quyết_định viết dịch_vụ web dự_định t...,[design]
4,đọc_bài viết băm mật_khẩu thaidn trường môn bả...,[security]


In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36092 entries, 0 to 36091
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   36092 non-null  object
 1   tags    36092 non-null  object
dtypes: object(2)
memory usage: 564.1+ KB


As you can see, *title* column contains titles of the posts and *tags* column contains the tags. It could be noticed that a number of tags for a post is not fixed and could be as many as necessary.

For a more comfortable usage, initialize *X_train*, *X_val*, *X_test*, *y_train*, *y_val*.

In [46]:
X_train, y_train = train['title'].values, train['tags'].values
X_val, y_val = validation['title'].values, validation['tags'].values
X_test = test['title'].values

In [47]:
X_train

array(['chào_mừng trở_lại blog duthaho tết_nhất dí this cố_gắng viết tất_niên dell chữ tiếng tiếng tiêu_đề trend tiêu_đề trang_web hàng lội blog follow nổi_tiếng github star tốn lọc viết bookmark lo_lắng điện_máy xanh lo quên duthaho lo build website https bookmark duthaho com a bookmark reading list for programmers bookmark đọc hàng bot chạy cơm tự_động update viết hót share reddit phản_hồi tích_cực pr trở_lại chủ_đề viết hôm_nay  github repo star hy_vọng có_ích 1 list of free learning resources https github comebookfoundationfree programming books tiêu_đề repo chứa resources tutorial article video ebook ngôn_ngữ free _lượng resource nổi đời đọc 2 free for developers https github comripienaarfree for dev repo repo chỗ sharing danh_sách công_cụ tool mục_đích công_việc lập_trình_viên code quantity ci cd security danh_sách tool chia thể_loại thử implement project tool thú_vị free repo 3 tech interview handbook https github comyangshuntech interview handbook repo hữu_ích đổi job tết no_nê

One of the most known difficulties when working with natural data is that it's unstructured. For example, if you use it "as is" and extract tokens just by splitting the titles by whitespaces, you will see that there are many "weird" tokens like *3.5?*, *"Flip*, etc. To prevent the problems, it's usually useful to prepare the data somehow.


In [48]:
import re
from underthesea import word_tokenize
from bs4 import BeautifulSoup

In [49]:
# REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
# BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# VIETNAMESE_DASH_STOPWORDS = set(stopwords.words('vietnamese_dash'))
VIETNAMESE_STOPWORDS = set(stopwords.words('vietnamese'))

def text_prepare(text):
    text = remove_html(text)
    text = convert_unicode(text)
#     text = chuan_hoa_dau_cau_tieng_viet(text)
    text = " ".join([s for s in text.split(" ") if s not in VIETNAMESE_STOPWORDS])
    text = word_tokenize(text, format='text')
    text = text.lower()
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ', text)
    # text = " ".join([s for s in text.split(" ") if s not in VIETNAMESE_DASH_STOPWORDS])
    text = re.sub(r'\s+', ' ', text).strip()

    # text = text.lower()# lowercase text
    # text = re.sub(REPLACE_BY_SPACE_RE," ",text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    # text = re.sub(BAD_SYMBOLS_RE,"",text)# delete symbols which are in BAD_SYMBOLS_RE from text
    # text = " ".join([s for s in text.split(" ") if s not in STOPWORDS])# delete stopwords from text
    # text = re.sub(' +', ' ', text)
    return text

### Pre-processing

#### Export to _vi_blog_train.tsv_

In [50]:
from tqdm import tqdm
from IPython.display import clear_output

In [51]:
def test_text_prepare():
    examples = ["hướng dẫn lập trình javascript, với nodejs",
                "Chương trình giới thiệu lập trình web"]
    answers = ["hướng lập_trình javascript nodejs", 
               "chương_trình giới_thiệu lập_trình web"]
              
              
    for ex, ans in zip(examples, answers):
        print(text_prepare(ex))
        if text_prepare(ex) != ans:
            return "Wrong answer for the case: '%s'" % ex
            
    return 'Basic tests are passed.'

In [52]:
print(test_text_prepare())

hướng lập_trình javascript nodejs
chương_trình giới_thiệu lập_trình web
Basic tests are passed.


Now we can preprocess the titles using function *text_prepare* and  making sure that the headers don't have bad symbols:

In [53]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [54]:
X_train[:3]

['chào_mừng trở_lại blog duthaho tết_nhất dí this cố_gắng viết tất_niên dell chữ tiếng tiếng tiêu_đề trend tiêu_đề trang_web hàng lội blog follow nổi_tiếng github star tốn lọc viết bookmark lo_lắng điện_máy xanh lo quên duthaho lo build website https bookmark duthaho com a bookmark reading list for programmers bookmark đọc hàng bot chạy cơm tự_động update viết hót share reddit phản_hồi tích_cực pr trở_lại chủ_đề viết hôm_nay github repo star hy_vọng có_ích 1 list of free learning resources https github comebookfoundationfree programming books tiêu_đề repo chứa resources tutorial article video ebook ngôn_ngữ free _lượng resource nổi đời đọc 2 free for developers https github comripienaarfree for dev repo repo chỗ sharing danh_sách công_cụ tool mục_đích công_việc lập_trình_viên code quantity ci cd security danh_sách tool chia thể_loại thử implement project tool thú_vị free repo 3 tech interview handbook https github comyangshuntech interview handbook repo hữu_ích đổi job tết no_nê học đi

For each tag and for each word calculate how many times they occur in the train corpus. 


In [55]:
y_train

array([list(['git']), list(['docker']), list(['javascript']), ...,
       list(['c++']), list(['c++']), list(['c++'])], dtype=object)

In [56]:
from nltk.tokenize.treebank import TreebankWordTokenizer
twd = TreebankWordTokenizer()
twd.tokenize(X_train[0])

['chào_mừng',
 'trở_lại',
 'blog',
 'duthaho',
 'tết_nhất',
 'dí',
 'this',
 'cố_gắng',
 'viết',
 'tất_niên',
 'dell',
 'chữ',
 'tiếng',
 'tiếng',
 'tiêu_đề',
 'trend',
 'tiêu_đề',
 'trang_web',
 'hàng',
 'lội',
 'blog',
 'follow',
 'nổi_tiếng',
 'github',
 'star',
 'tốn',
 'lọc',
 'viết',
 'bookmark',
 'lo_lắng',
 'điện_máy',
 'xanh',
 'lo',
 'quên',
 'duthaho',
 'lo',
 'build',
 'website',
 'https',
 'bookmark',
 'duthaho',
 'com',
 'a',
 'bookmark',
 'reading',
 'list',
 'for',
 'programmers',
 'bookmark',
 'đọc',
 'hàng',
 'bot',
 'chạy',
 'cơm',
 'tự_động',
 'update',
 'viết',
 'hót',
 'share',
 'reddit',
 'phản_hồi',
 'tích_cực',
 'pr',
 'trở_lại',
 'chủ_đề',
 'viết',
 'hôm_nay',
 'github',
 'repo',
 'star',
 'hy_vọng',
 'có_ích',
 '1',
 'list',
 'of',
 'free',
 'learning',
 'resources',
 'https',
 'github',
 'comebookfoundationfree',
 'programming',
 'books',
 'tiêu_đề',
 'repo',
 'chứa',
 'resources',
 'tutorial',
 'article',
 'video',
 'ebook',
 'ngôn_ngữ',
 'free',
 '_lượng',

In [57]:
# Dictionary of all tags from train corpus with their counts.
tags_counts = {}
# Dictionary of all words from train corpus with their counts.
words_counts = {}

all_tags = []
vocabulary=[]
from collections import Counter
from nltk.tokenize.treebank import TreebankWordTokenizer
twd = TreebankWordTokenizer()

for tag in y_train:
    all_tags.extend(tag)

for txt in X_train:
    vocabulary.extend(twd.tokenize(txt))

tags_counts = Counter(all_tags)
words_counts = Counter(vocabulary)


We are assuming that *tags_counts* and *words_counts* are dictionaries like `{'some_word_or_tag': frequency}`. After applying the sorting procedure, results will be look like this: `[('most_popular_word_or_tag', frequency), ('less_popular_word_or_tag', frequency), ...]

### Transforming text to a vector

Machine Learning algorithms work with numeric data and we cannot use the provided text data "as is". There are many ways to transform text data to numeric vectors. In this task you will try to use two of them.

#### Bag of words

One of the well-known approaches is a *bag-of-words* representation. To create this transformation, follow the steps:
1. Find *N* most popular words in train corpus and numerate them. Now we have a dictionary of the most popular words.
2. For each title in the corpora create a zero vector with the dimension equals to *N*.
3. For each text in the corpora iterate over words which are in the dictionary and increase by 1 the corresponding coordinate.

Let's try to do it for a toy example. Imagine that we have *N* = 4 and the list of the most popular words is 

    ['hi', 'you', 'me', 'are']

Then we need to numerate them, for example, like this: 

    {'hi': 0, 'you': 1, 'me': 2, 'are': 3}

And we have the text, which we want to transform to the vector:

    'hi how are you'

For this text we create a corresponding zero vector 

    [0, 0, 0, 0]
    
And iterate over all words, and if the word is in the dictionary, we increase the value of the corresponding position in the vector:

    'hi':  [1, 0, 0, 0]
    'how': [1, 0, 0, 0] # word 'how' is not in our dictionary
    'are': [1, 0, 0, 1]
    'you': [1, 1, 0, 1]

The resulting vector will be 

    [1, 1, 0, 1]
   
Implement the described encoding in the function *my_bag_of_words* with the size of the dictionary equals to 5000. To find the most common words use train data. You can test your code using the function *test_my_bag_of_words*.

In [58]:
import operator

TOP_WORDS=sorted_d = sorted(words_counts.items(), key=operator.itemgetter(1),reverse=True)[:5000]

In [59]:
TOP_WORDS

[('1', 151797),
 ('2', 88055),
 ('0', 79348),
 ('a', 74610),
 ('class', 64306),
 ('the', 59795),
 ('i', 59405),
 ('file', 56452),
 ('code', 52472),
 ('3', 50603),
 ('viết', 50478),
 ('function', 47789),
 ('this', 45351),
 ('return', 41457),
 ('to', 41177),
 ('ví_dụ', 41054),
 ('name', 40946),
 ('ta', 40744),
 ('dữ_liệu', 40388),
 ('ứng_dụng', 39987),
 ('int', 39379),
 ('n', 38067),
 ('user', 37670),
 ('test', 37661),
 ('end', 35936),
 ('chạy', 35833),
 ('if', 35345),
 ('hàm', 34896),
 ('in', 33793),
 ('app', 32688),
 ('thử', 31413),
 ('div', 31156),
 ('for', 30498),
 ('4', 30361),
 ('5', 28955),
 ('id', 28481),
 ('data', 27806),
 ('https', 27581),
 ('and', 27051),
 ('js', 25976),
 ('new', 25536),
 ('gọi', 25279),
 ('x', 25260),
 ('server', 25164),
 ('is', 25131),
 ('of', 24805),
 ('kiểm_tra', 24658),
 ('string', 24626),
 ('c', 23055),
 ('component', 22702),
 ('object', 22586),
 ('giúp', 22456),
 ('lỗi', 22285),
 ('public', 22115),
 ('html', 22100),
 ('http', 22038),
 ('kết_quả', 21714)

In [60]:
import operator
DICT_SIZE = 5000
ALL_WORDS =[i for i,j in sorted(words_counts.items(), key=operator.itemgetter(1),reverse=True)[:5000]]

WORDS_TO_INDEX = {}
for count, word in enumerate(ALL_WORDS):
    WORDS_TO_INDEX.update({word:count})
    
INDEX_TO_WORDS = dict((v,k) for k,v in WORDS_TO_INDEX.items())
#ALL_WORDS = WORDS_TO_INDEX.keys()

def my_bag_of_words(text, words_to_index, dict_size):

    result_vector = np.zeros(dict_size)
    
    for word in text.split():
        if word in words_to_index.keys():
            x= words_to_index[word]
            result_vector[x]=1
    return result_vector

In [61]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    for ex, ans in zip(examples, answers):
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [62]:
print(test_my_bag_of_words())

Basic tests are passed.


Now apply the implemented function to all samples (this might take up to a minute):

In [63]:
from scipy import sparse as sp_sparse

In [64]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (36092, 5000)
X_val shape  (613, 5000)
X_test shape  (1498, 5000)


As you might notice, we transform the data to sparse representation, to store the useful information efficiently. There are many [types](https://docs.scipy.org/doc/scipy/reference/sparse.html) of such representations, however sklearn algorithms can work only with [csr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) matrix, so we will use this one.

In [65]:
X_train_mybag.shape

(36092, 5000)

In [66]:
row = X_train_mybag[10].toarray()[0]
row

array([1., 1., 0., ..., 0., 0., 0.])

In [67]:
row = X_train_mybag[10].toarray()[0]
non_zero_elements_count = len(row[row != 0])


#### TF-IDF

The second approach extends the bag-of-words framework by taking into account total frequencies of words in the corpora. It helps to penalize too frequent words and provide better features space. 

Implement function *tfidf_features* using class [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) from *scikit-learn*. Use *train* corpus to train a vectorizer. Don't forget to take a look into the arguments that you can pass to it. We suggest that you filter out too rare words (occur less than in 5 titles) and too frequent words (occur more than in 90% of the titles). Also, use bigrams along with unigrams in your vocabulary. 

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [69]:
def tfidf_features(X_train, X_val, X_test):
   
    
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    
    
    tfidf_vectorizer = TfidfVectorizer(token_pattern = '(\S+)') # we want c++ or c# to be there
    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)

    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

Once you have done text preprocessing, always have a look at the results. Be very careful at this step, because the performance of future models will drastically depend on it. 

In this case, check whether you have c++ or c# in your vocabulary, as they are obviously important tokens in our tags prediction task:

In [70]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

In [71]:
X_val_tfidf

<613x298322 sparse matrix of type '<class 'numpy.float64'>'
	with 122874 stored elements in Compressed Sparse Row format>

In [72]:
'c++' in tfidf_vocab # however we need to it,
 #Get back to the function above and use '(\S+)' regexp as a token_pattern 
#in the constructor of the vectorizer.

False

### MultiLabel classifier

As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose it is convenient to use [MultiLabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) from *sklearn*.

In [73]:
from sklearn.preprocessing import MultiLabelBinarizer

In [74]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

Implement the function *train_classifier* for training a classifier. In this task we suggest to use One-vs-Rest approach, which is implemented in [OneVsRestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) class. In this approach *k* classifiers (= number of tags) are trained. As a basic classifier, use [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time, because a number of classifiers to train is large.

In [75]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [76]:
def train_classifier(X_train, y_train):
    
    model = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter=10000)).fit(X_train, y_train)
    
    return model
        

Train the classifiers for different data transformations: *bag-of-words* and *tf-idf*.

In [77]:
classifier_mybag = train_classifier(X_train_mybag, y_train)
classifier_tfidf = train_classifier(X_train_tfidf, y_train)

C:\Users\Christian\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Christian\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Now you can create predictions for the data. You will need two types of predictions: labels and scores.

In [78]:
from joblib import dump, load
dump(classifier_tfidf, 'tfidf_model.joblib')
dump(classifier_mybag, 'bag_model.joblib')

['bag_model.joblib']

In [79]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

Now take a look at how classifier, which uses TF-IDF, works for a few examples:

In [80]:
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_tfidf)
y_val_inversed = mlb.inverse_transform(y_val)

for i in range(300):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_val[i],
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

Title:	ruby on rails cung_cấp module activerecord xây_dựng hàng_loạt phương_thức giúp thao_tác cơ_sở dữ_liệu dễ_dàng lưu_ý xử dụng phương_thức tham_số param client gửi lỗ_hổng khai_thác lỗi sql injection ta gem brakeman giúp phát_hiện truy_vấn lỗi chủ_động viết code an_toàn phương_thức lỗi sql injection phương_thức tính_toán average calculate count maximum minimum sum ví_dụ params column point where users id 1 user calculate sum params column select sum age from users where id 1 from orders câu_lệnh thay_vì tổng_số user câu_lệnh tổng user xác_định id 1 tương_tự phương_thức phương_thức delete_all phương_thức xóa ghi cẩn_thận phương_thức delete_all tham_số tìm_kiếm phương_thức find tham_số string array hash array hash an_toàn truyền tham_số nhập user ví_dụ câu_lệnh xóa toàn_bộ user params id 1 or 1 1 user delete_all id params id delete from users where id 1 or 1 1 phương_thức destroy_all phương_thức destroy_all an_toàn phương_thức delete_all một_chút gọi call_back model quan_hệ model tươ

Now, we would need to compare the results of different predictions, e.g. to see whether TF-IDF transformation helps or to try different regularization techniques in logistic regression. For all these experiments, we need to setup evaluation procedure. 

### Evaluation

To evaluate the results we will use several classification metrics:
 - [Accuracy](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
 - [F1-score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)
 - [Area under ROC-curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
 - [Area under precision-recall curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score) 
 

In [81]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

Implement the function *print_evaluation_scores* which calculates and prints to stdout:
 - *accuracy*
 - *F1-score macro/micro/weighted*
 - *Precision macro/micro/weighted*

In [82]:
def print_evaluation_scores(y_val, predicted):
    
    print("Accuracy",accuracy_score(y_val, predicted))
    print("F1 Score",f1_score(y_val, predicted, average='macro'))
    print("Average Precision",average_precision_score(y_val, predicted, average='macro') )
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

In [83]:
print('Bag-of-words')
print_evaluation_scores(y_val, y_val_predicted_labels_mybag)
print('Tfidf')
print_evaluation_scores(y_val, y_val_predicted_labels_tfidf)

Bag-of-words
Accuracy 0.5301794453507341
F1 Score 0.41997292203373016
Average Precision nan
Tfidf
Accuracy 0.5497553017944535
F1 Score 0.36760220232918817
Average Precision nan


C:\Users\Christian\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Christian\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Christian\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\Christian\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Christian\AppData\Roaming\Python\Python36\site-package

You might also want to plot some generalization of the [ROC curve](http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc) for the case of multi-label classification. Provided function *roc_auc* can make it for you. The input parameters of this function are:
 - true labels
 - decision functions scores
 - number of classes

In [191]:
from sklearn.metrics import roc_auc_score
import numpy as np
%matplotlib inline

In [137]:
n_classes = len(tags_counts)
print(n_classes)
# roc_auc_score(y_val, y_val_predicted_scores_mybag, n_classes)
# roc_auc_score(np.array(y_val), np.array(y_val_predicted_scores_mybag))

3226


In [138]:
n_classes = len(tags_counts)
# roc_auc_score(y_val, y_val_predicted_scores_tfidf, n_classes)
roc_auc_score(y_val, y_val_predicted_scores_tfidf)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.